# EuroPython 2015 program grid

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

In [ ]:
import json
import datetime as dt
from random import choice, randrange, shuffle
from copy import deepcopy

from functools import partial
from operator import itemgetter

from IPython.display import display, HTML
show = lambda s: display(HTML(s))

In [ ]:
talk_sessions = json.load(open('talk_abstracts.json'))

In [ ]:
list(talk_sessions.keys())

In [ ]:
#all talks
all_talks = []
for s in talk_sessions.values():
    all_talks.extend(list(s.values()))

#the talks worth for scheduling
grid_talks = []
sessions = talk_sessions.copy()
general_grid_sessions = ['Talks', 'EuroPython sessions', 'Other sessions', 'Trainings']
for session_name in general_grid_sessions:
    grid_talks.extend(sessions[session_name].values())
    
for talk in grid_talks:
    talk.pop('abstracts')

In [ ]:
import datetime
from   collections import OrderedDict
from   functools   import partial

# talk rooms
# ((h, m), duration)
type1_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 45),
                  ((14, 30), 45),
                  ((15, 15), 30),
                  ((15, 45), 30),
                  ((17,  0), 30)]

type2_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 45),
                  ((14, 30), 45),
                  ((15, 15), 60),
                  ((17,  0), 30)]

type3_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 30),
                  ((14, 30), 45),
                  ((15, 15), 30),
                  ((15, 45), 30),
                  ((17,  0), 30)]

#tutorials
type4_schedule = [((11,  0), 150),
                  ((13, 30), 180)]

keynote_time   = (( 9, 30), 60)
lightning_time = ((17, 30), 60) 
lunch_time     = ((13,  0), 90) 
am_coffee_time = ((10, 30), 30)
pm_coffee_time = ((16, 15), 45) 

breaks = [(keynote_time,   'Keynote speech'),
          (lunch_time,     'Lunch'),
          (am_coffee_time, 'Coffee break'),
          (pm_coffee_time, 'Coffee break'),
          (lightning_time, 'Lightning talks')]

daily_timegrid = lambda schedule: OrderedDict([(datetime.time(*slot[0]), slot[1]) for slot in schedule])

room1_schedule = daily_timegrid(type1_schedule) # A1, the google room
room2_schedule = daily_timegrid(type2_schedule) # A3, pythonanywhere room
room3_schedule = daily_timegrid(type3_schedule) # A2
room4_schedule = daily_timegrid(type3_schedule) # Barria1
room5_schedule = daily_timegrid(type3_schedule) # Barria2

room6_schedule = daily_timegrid(type4_schedule) # Room E
room7_schedule = daily_timegrid(type4_schedule) # Room A4

daily_schedule = OrderedDict([('A1' , room1_schedule), 
                              ('A3' , room2_schedule),
                              ('A2' , room3_schedule),
                              ('Ba1', room4_schedule),
                              ('Ba2', room5_schedule),
                              ('E'  , room6_schedule),
                              ('A4' , room7_schedule)])

# slots_durations
slots_durations = [list(slots.values()) for slots in list(daily_schedule.values())]
slots = [val for sublist in slots_durations for val in sublist]

In [ ]:
def find_talk(talk_title):
    talks = []
    for talk in all_talks:
        if talk_title in talk['title']:
            talks.append(talk)
    return talks

find_talk('Python and PyPy performance')

In [ ]:
tags = {}
text = []
for talk in all_talks:
    for t in talk['tags']:
        if t.lower() == 'python':
            continue
        if not t.lower() in tags:
            tags[t.lower()] = 0
        tags[t.lower()] += 1
        text.append(t.lower())
tags_sorted = sorted(tags.items(), key=itemgetter(1), reverse=True)

In [ ]:
def pick_talk(talks, tags, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    while tags:
        randidx = randrange(0, len(tags))
        atag    = tags.pop(randidx)[0]
        for tlkidx, talk in enumerate(talks):
            talk_tags = [t.lower() for t in talk['tags']]
            if talk['duration'] == duration and atag in talk_tags:
                atalk = talks.pop(tlkidx)
                return atalk

    return pick_talk_by_duration(talks, duration)


def pick_talk_by_duration(talks, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    for tlkidx, talk in enumerate(talks):
        if talk['duration'] == duration:
                atalk = talks.pop(tlkidx)
                return atalk
    return {}

In [ ]:
talks = grid_talks.copy()
shuffle(talks)

# random pick talks
week_talks = []
n_days = 5
for d in range(n_days):
    day_talks = []
    for slot in slots:
        day_tags = tags_sorted.copy()
        try:
            day_talks.append(pick_talk(talks, day_tags, slot))
        except:
            print('No talks left.')
                    
    week_talks.append(day_talks)

In [ ]:
weekly_schedule = OrderedDict()

for d in range(n_days):
    weekly_schedule[d] = deepcopy(daily_schedule)
    day_talks          = week_talks[d].copy()
    for room in daily_schedule:
        for slot in daily_schedule[room]:
            #print(weekly_schedule[d][room].keys())
            weekly_schedule[d][room][slot] = day_talks.pop(0)
            #print(day_talks)

In [ ]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


def make_daily_schedule(time_list, header=''):
    table = ListTable()
    table.append(header)
    for slot in time_list:
        table.append([slot] + time_list[slot])
    return table


def get_room_schedule(weekly_schedule, room_name):
    slots = list(daily_schedule[room_name].keys())
    daily_slots = []
    for slot in slots:
        talks = [weekly_schedule[d][room_name][slot].get('id', '-') for d in range(n_days)]
        daily_slots.append((slot, talks))
    room_schedule = OrderedDict(daily_slots)
    return room_schedule


weekday_names = ['Monday, June 20th', 
                 'Tuesday, June 21st',
                 'Wednesday, June 22nd',
                 'Thursday, June 23rd',
                 'Friday, June 24th']

In [ ]:
show('<h1>Non-schedules talks</h1>')
show('<p>Move this to a 1-hour slot instead of the EPS meeting</p>')

for talk in talks:
    print(talks)

# TALKS

### The Google Room

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A1'), header=['Google Room'] + weekday_names)

### Room A2

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A2'), header=['A2'] + weekday_names)

### The PythonAnywhere Room

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A3'), header=['PythonAnywhere Room'] + weekday_names)

### Room Barria1

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'Ba1'), header=['Barria1 Room'] + weekday_names)

### Room Barria2

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'Ba2'), header=['Barria2 Room'] + weekday_names)

# TRAININGS

### Room E

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'E'), header=['E Room'] + weekday_names)

### Room A4

In [ ]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A4'), header=['A4 Room'] + weekday_names)